In [34]:
from collections import deque, defaultdict

class AhoCorasick(object):
    def __init__(self, dictionary, values):
        self.build_trie(dictionary, values)
        self.build_fails()

    def build_trie(self, dictionary, values):
        self.trie = trie = [{}]
        self.outputs = outputs = defaultdict(dict)  # state => index => value
        state = 0
        for index, word in enumerate(dictionary):
            state = 0
            for char in word:
                if char not in trie[state]:
                    new = len(trie)
                    trie[state][char] = new
                    trie.append({})
                state = trie[state][char]
            outputs[state][index] = values[index]  # Avoid checking for state with defaultdict
    
    def build_fails(self):
        # Avoid . lookups
        trie = self.trie
        suffixes = self.suffixes
        outputs = self.outputs
        
        self.fails = fails = { state: 0 for state in trie[0].values() }
        self.memoize_output = memoize_output = defaultdict(list)
        
        # states = [(parent, char, state)]
        states = deque([(0, c, s) for c, s in trie[0].items()])  # BFS
        pop_state = states.popleft
        push_state = states.append
        while states:
            parent, char, state = pop_state()
            suffix_found = False
            for parent_suffix in suffixes(parent):
                # Follow fails until one has a child with same letter
                try:  # This is called many, many times. .get() is too slow
                    suffix = trie[parent_suffix][char]
                except KeyError:
                    continue
                if not suffix_found:
                    fails[state] = suffix
                    suffix_found = True
                if suffix in outputs:
                    memoize_output[state].append(suffix)  # Avoid checking for state with defaultdict
            if state not in fails:
                fails[state] = 0  # Default to fail to root
            for char, next_state in trie[state].items():
                push_state((state, char, next_state))
    
    def suffixes(self, state):
        fails = self.fails  # Local variable for performance
        try:  # .get() too slow
            suffix = fails[state]
        except KeyError:
            return
        while True:
            yield suffix
            try:
                suffix = fails[suffix]
            except KeyError:
                return
    
    def goto(self, state, letter):
        next_state = 0  # default to root
        try:  # try child
            next_state = self.trie[state][letter]
        except KeyError:
            # try suffix children
            for suffix in self.suffixes(state):
                try:
                    next_state = self.trie[suffix][letter]
                except KeyError:
                    continue
                break
        return next_state
    
    def values(self, state, low_index, high_index):
        outputs = [state] if state in self.outputs else []
        if state in self.memoize_output:
            outputs.extend(self.memoize_output[state])
        for output in outputs:
            for index, value in self.outputs[output].items():
                if low_index <= index <= high_index:
                    yield value

    def value(self, string, low_index, high_index):
        state = 0
        val = 0
        for c in string:
            state = self.goto(state, c)
            for v in self.values(state, low_index, high_index):
                val += v
        return val

aho = AhoCorasick(['a', 'ab', 'bab', 'bc', 'bca', 'c', 'caa'], [1, 1, 1, 1, 1, 1, 1])
assert aho.fails == {1: 0, 3: 0, 8: 0, 2: 3, 4: 1, 6: 8, 9: 1, 5: 2, 7: 9, 10: 1}
#print(aho.memoize_output)
assert aho.memoize_output == {4: [1], 6: [8], 9: [1], 5: [2], 7: [1], 10: [1]}
print(aho.value('abccab', 0, 0))

aho = AhoCorasick(['he', 'she', 'his', 'hers'], [1, 1, 1, 1])
assert aho.fails == {1: 0, 3: 0, 2: 0, 6: 0, 4: 1, 8: 0, 7: 3, 5: 2, 9: 3}
print(aho.value('ahishers', 0, 3))

2
4


In [28]:
def counts(gene, dna):
    for i in range(len(dna)):
        yield dna[i:i+len(gene)] == gene

def dna_health(health, genes, first, last, dna):
    return sum([
        sum(counts(genes[i], dna)) * health[i]
        for i in range(first, last+1)
    ])
int()

0

In [31]:
class Node(object):
    def __init__(self, path, character, parent, suffix=None):
        self.path = path
        self.character = character
        self.parent = parent
        self.suffix = suffix
        self.health = None
    
    def __repr__(self):
        return '<Node character:{}, parent:{}, suffix:{} >'.format(
            self.character,
            '--' if not self.parent else '<Node path:'+self.parent.path+'>',
            '--' if not self.suffix else '<Node path:'+self.suffix.path+'>'
        )

In [72]:
class Node(object):
    def __init__(self, path, value, parent):
        self.path = path
        self.children = {}
        self.value = value
        self.parent = parent
        self.suffix = None
        self.dict_suffix = None

    def __getitem__(self, letter):
        return self.children.get(letter)
    
    def __contains__(self, letter):
        return letter in self.children
    
    def __setitem__(self, letter, node):
        self.children[letter] = node
    
    def __repr__(self):
        return '{}\t\t{}\t\t{}\t\t{}\n'.format(
            "'"+self.path+"'",
            self.value,
            '--' if not self.suffix else "'"+self.suffix.path+"'",
            '--' if not self.dict_suffix else "'"+self.dict_suffix.path+"'"
        ) + ''.join([
            child.__repr__() 
            for child in self.children.values()
        ])


class Trie(object):
    def __init__(self, dictionary, values):
        self.trie_root = Node('', 0, None)

        for index, gene in enumerate(dictionary):
            curr_node = self.trie_root
            for i, letter in enumerate(gene):
                in_dict = i == len(gene) - 1  # full gene
                node_value = values[index] if in_dict else 0
                if letter not in curr_node:
                    in_dict = i == len(gene) - 1
                    curr_node[letter] = Node(gene[:i+1], node_value, curr_node)
                else:
                    curr_node[letter].value += node_value
                curr_node = curr_node[letter]
        
        self._put_suffixes(self.trie_root, '')
        self._put_dict_suffixes(self.trie_root)

    def value(self, haystack):
        total = 0
        node = self.trie_root
        for straw in haystack:
            node = self._next_good_node(node, straw)
            if node.value:
                total += node.value
            total += sum(self._value_dict_suffixes(node))
        return total
    
    def _value_dict_suffixes(self, node):
        suffix = node.dict_suffix
        while suffix:
            yield suffix.value
            suffix = suffix.dict_suffix
    
    def _next_good_node(self, node, letter):
        if node[letter]:
            return node[letter]
        if not node.suffix:
            return self.trie_root[letter] if self.trie_root[letter] else self.trie_root
        cursor = node.suffix
        while cursor != self.trie_root and not cursor[letter]:
            cursor = cursor.suffix if cursor.suffix else self.trie_root
        return cursor[letter] if cursor[letter] else self.trie_root

    def _put_suffixes(self, node, letter):
        cursor = node.parent
        while cursor:
            suffix = cursor[letter]
            if suffix and suffix != node:
                node.suffix = suffix
                break
            elif cursor == self.trie_root:
                node.suffix = self.trie_root
                break
            cursor = cursor.suffix
        for key, child in node.children.items():
            self._put_suffixes(child, key)
    
    def _put_dict_suffixes(self, node):
        cursor = node.suffix
        while cursor:
            if cursor.value:
                node.dict_suffix = cursor
            cursor = cursor.suffix
        for child in node.children.values():
            self._put_dict_suffixes(child)

    def __repr__(self):
       return self.trie_root.__repr__()

trie = Trie(['a', 'ab', 'bab', 'bc', 'bca', 'c', 'caa'], [1, 1, 1, 1, 1, 1, 1])
trie.value('abccab')

trie = Trie(['hlimdfbfnv', 'mguznrcpfc', 'nrmweeookb', 'drolersfwh', 'ckpykeqotx', 'sioefulviv', 'wnmkasbuzz', 
             'ddkscwwukr', 'rfzhjgbwbl', 'rzagjaymua', 'mxdyrhunbg', 'eulfdxogtr', 'rnrrtctrpp', 'tdmzbfgxsi', 
             'fyereiquol', 'liyebrhvly', 'kgbzfeembz', 'wgxazdirzx', 'flfdrgxydi', 'woqzpwdvkg', 'ugpuiqxrix', 
             'qnkxsbfpcj', 'zazqaqmdly', 'mgebaorzfz', 'yxoiuhmayo', 'lyqkoqacwn', 'aivgjxucxc', 'cxzsgwbuya', 
             'klyavotxsp', 'muzickfwmc', 'aqccjiakey', 'mojmqgajfu', 'yrozzqjfpw', 'jrmltxvtkz', 'twpejgmlpr', 
             'gqlwpknbre', 'xdvlqplmkv', 'ngtfmelzsc', 'qyudukojnh', 'nkmjxdairm', 'fgublhhygz', 'byxvcuhsdu', 
             'btgocgreqk', 'syqnzeuicc', 'ifahdebmwh', 'jaapoexhio', 'rcmjpnnlxq', 'nfvonauqnt', 'xwtznjdlqn', 
             'bjqnshcgtz', 'yghvwuwrml', 'kmhdlumrhe', 'einwxhebpx', 'bnfilcejts', 'ufebiqxwjh', 'cnprmnysoq', 
             'rrfwbqahzv', 'atagwkwwif', 'dkvsbjhcby', 'surxqvqter', 'oenpljzjhi', 'rkuofwxoaa', 'osugrmdjfh', 
             'bwoolbzmkh', 'wdtrrypqpp', 'qdjmlcbomi', 'wpekdpleex', 'nabhtuhinw', 'zfcksnntcb', 'dyqiktzxzd', 
             'ungxuzubkh', 'almcwgrlbt', 'mftcndxoaw', 'sxjawdzshl', 'zjxonvwegy', 'ysfruuxtiz', 'payzavecpn', 
             'ppwofjjbop', 'bojghfaeyj', 'golgpodtst', 'hhifwprhqf', 'xuvgacodjm', 'orcbxrpbnj', 'uwtebrtsyl', 
             'zxfugizuli', 'gzzjawcszp', 'btnwxrnqlm'],
            [1803633, 3357388, 1112112, 5656776, 4438527, 3841975, 4102090, 2113339, 2977711, 1709727, 1666821, 
             4167887, 3742911, 1948785, 3057238, 1940358, 4574138, 4598641, 2922682, 1839758, 4562812, 1508583, 
             2531144, 3192788, 4971388, 3448060, 2579952, 4895338, 5133938, 5667253, 5417655, 3453923, 3024642, 
             5529768, 3110699, 3979521, 5888095, 3729142, 2609212, 2865806, 1955221, 3276034, 3550045, 4698132, 
             1741171, 5607283, 5638490, 2831662, 1722277, 2561172, 3671420, 5285089, 5586108, 5202564, 4994229, 
             2073848, 5166977, 1574181, 5969186, 1817267, 1241435, 2903194, 1787542, 5782429, 4949314, 1414593, 
             1278302, 4837409, 4143735, 2887514, 4219567, 5098956, 2679900, 1769612, 1313440, 5937424, 3893247, 
             3468283, 2769086, 2131876, 5029455, 2956858, 3933318, 2131915, 4675774, 2927547, 3205764])
#print(trie)
trie.value('zjxonvwegyeulfdxogtr')

trie = Trie(['mguznrcpfc', 'nrmweeookb', 'drolersfwh', 'ckpykeqotx', 'sioefulviv', 'wnmkasbuzz', 'ddkscwwukr', 
             'rfzhjgbwbl', 'rzagjaymua', 'mxdyrhunbg', 'eulfdxogtr', 'rnrrtctrpp', 'tdmzbfgxsi', 'fyereiquol', 
             'liyebrhvly', 'kgbzfeembz', 'wgxazdirzx', 'flfdrgxydi', 'woqzpwdvkg', 'ugpuiqxrix', 'qnkxsbfpcj', 
             'zazqaqmdly', 'mgebaorzfz', 'yxoiuhmayo', 'lyqkoqacwn', 'aivgjxucxc', 'cxzsgwbuya', 'klyavotxsp', 
             'muzickfwmc', 'aqccjiakey', 'mojmqgajfu', 'yrozzqjfpw', 'jrmltxvtkz', 'twpejgmlpr', 'gqlwpknbre', 
             'xdvlqplmkv', 'ngtfmelzsc', 'qyudukojnh', 'nkmjxdairm', 'fgublhhygz', 'byxvcuhsdu', 'btgocgreqk', 
             'syqnzeuicc', 'ifahdebmwh', 'jaapoexhio', 'rcmjpnnlxq', 'nfvonauqnt', 'xwtznjdlqn', 'bjqnshcgtz', 
             'yghvwuwrml', 'kmhdlumrhe', 'einwxhebpx', 'bnfilcejts', 'ufebiqxwjh', 'cnprmnysoq', 'rrfwbqahzv', 
             'atagwkwwif', 'dkvsbjhcby', 'surxqvqter', 'oenpljzjhi', 'rkuofwxoaa', 'osugrmdjfh', 'bwoolbzmkh', 
             'wdtrrypqpp'],
            [3357388, 1112112, 5656776, 4438527, 3841975, 4102090, 2113339, 2977711, 1709727, 1666821, 4167887, 
             3742911, 1948785, 3057238, 1940358, 4574138, 4598641, 2922682, 1839758, 4562812, 1508583, 2531144, 
             3192788, 4971388, 3448060, 2579952, 4895338, 5133938, 5667253, 5417655, 3453923, 3024642, 5529768, 
             3110699, 3979521, 5888095, 3729142, 2609212, 2865806, 1955221, 3276034, 3550045, 4698132, 1741171, 
             5607283, 5638490, 2831662, 1722277, 2561172, 3671420, 5285089, 5586108, 5202564, 4994229, 2073848, 
             5166977, 1574181, 5969186, 1817267, 1241435, 2903194, 1787542, 5782429, 4949314])

trie.value('bjqnshcgtzugpuiqxrix')

7123984

In [38]:
import time
with open('input4.txt') as inputs:
    n = int(inputs.readline())
    genes = inputs.readline().rstrip().split()
    health = list(map(int, inputs.readline().rstrip().split()))
    s = int(inputs.readline())

    least = float('inf')
    most = float('-inf')
    
    aho = AhoCorasick(genes, health)

    for s_itr in range(s):
        t = time.time()
        firstLastd = inputs.readline().split()

        first = int(firstLastd[0])
        last = int(firstLastd[1])
        dna = firstLastd[2]

        score = aho.value(dna, first, last)
        #score_old = dna_health(health, genes, first, last, dna)

        #if score != score_old:
        #    print('wrong')

        if score < least:
          least = score
        if score > most:
          most = score

    print(least, most)

15806635 20688978289


In [47]:
'0 7353994' == '0 7353994'

True

building trie from 100000 genes
build_trie took 23.61309003829956
build_fails took 101.95170903205872
memoize_dict took 217.46535897254944
         40528562 function calls in 118.872 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       15    0.000    0.000    0.000    0.000 :0(acquire)
  3784055    5.573    0.000    5.573    0.000 :0(append)
        1    0.000    0.000  118.872  118.872 :0(exec)
 15707174   21.917    0.000   21.917    0.000 :0(get)
       12    0.000    0.000    0.000    0.000 :0(getpid)
       12    0.000    0.000    0.000    0.000 :0(isinstance)
  1261350    2.093    0.000    2.093    0.000 :0(len)
  2522700    3.492    0.000    3.492    0.000 :0(popleft)
        3    0.000    0.000    0.001    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        6    0.000    0.000    0.000    0.000 :0(time)
  2522703    3.578    0.000    3.578    0.000 :0(values)
        1    5.664    5.664    9.811    9.811 <ipython-input-3-52f3bf81878a>:19(build_trie)
        1   16.974   16.974   37.084   37.084 <ipython-input-3-52f3bf81878a>:39(build_fails)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-52f3bf81878a>:40(<dictcomp>)
        1    0.000    0.000  118.678  118.678 <ipython-input-3-52f3bf81878a>:5(__init__)
        1   22.676   22.676   71.782   71.782 <ipython-input-3-52f3bf81878a>:56(memoize_dict)
 14730397   36.710    0.000   55.019    0.000 <ipython-input-3-52f3bf81878a>:70(suffixes)
        1    0.194    0.194  118.872  118.872 <string>:1(<module>)
       15    0.000    0.000    0.001    0.000 iostream.py:195(schedule)
       12    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
       12    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
       12    0.000    0.000    0.001    0.000 iostream.py:382(write)
       15    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        1    0.000    0.000  118.872  118.872 profile:0(AhoCorasick(genes[first:last+1], health[first:last+1]))
        0    0.000             0.000          profile:0(profiler)
       15    0.000    0.000    0.000    0.000 socket.py:334(send)
       15    0.000    0.000    0.000    0.000 threading.py:1038(_wait_for_tstate_lock)
       15    0.000    0.000    0.000    0.000 threading.py:1080(is_alive)
       15    0.000    0.000    0.000    0.000 threading.py:507(is_set)

combining build_fails and memoize_dict to not have to call suffixes twice for each state

building trie from 100000 genes
build_trie took 23.82583713531494
build_fails took 304.7511749267578
         38232678 function calls in 118.262 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       14    0.000    0.000    0.000    0.000 :0(acquire)
  2522709    4.051    0.000    4.051    0.000 :0(append)
        1    0.000    0.000  118.262  118.262 :0(exec)
 20863378   31.085    0.000   31.085    0.000 :0(get)
       12    0.000    0.000    0.000    0.000 :0(getpid)
       12    0.000    0.000    0.000    0.000 :0(isinstance)
  1261350    2.123    0.000    2.123    0.000 :0(len)
  1261350    1.937    0.000    1.937    0.000 :0(popleft)
        3    0.000    0.000    0.001    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        6    0.000    0.000    0.000    0.000 :0(time)
  1261352    1.982    0.000    1.982    0.000 :0(values)
        1    5.683    5.683    9.910    9.910 <ipython-input-8-859901f0482f>:19(build_trie)
        1   42.427   42.427  108.153  108.153 <ipython-input-8-859901f0482f>:39(build_fails)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-859901f0482f>:41(<dictcomp>)
        1    0.000    0.000  118.063  118.063 <ipython-input-8-859901f0482f>:5(__init__)
 11062364   28.774    0.000   43.705    0.000 <ipython-input-8-859901f0482f>:78(suffixes)
        1    0.198    0.198  118.261  118.261 <string>:1(<module>)
       14    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
       12    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
       12    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
       12    0.000    0.000    0.001    0.000 iostream.py:382(write)
       14    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        1    0.000    0.000  118.262  118.262 profile:0(AhoCorasick(genes[first:last+1], health[first:last+1]))
        0    0.000             0.000          profile:0(profiler)
       14    0.000    0.000    0.000    0.000 socket.py:334(send)
       14    0.000    0.000    0.000    0.000 threading.py:1038(_wait_for_tstate_lock)
       14    0.000    0.000    0.000    0.000 threading.py:1080(is_alive)
       14    0.000    0.000    0.000    0.000 threading.py:507(is_set)

not calling .get when getting suffix during build_fails

building trie from 100000 genes
build_trie took 23.812018871307373
build_fails took 224.72711062431335
         28431609 function calls in 88.859 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.000    0.000 :0(acquire)
  2522705    4.036    0.000    4.036    0.000 :0(append)
        1    0.000    0.000   88.859   88.859 :0(exec)
 11062364   15.042    0.000   15.042    0.000 :0(get)
        8    0.000    0.000    0.000    0.000 :0(getpid)
        8    0.000    0.000    0.000    0.000 :0(isinstance)
  1261350    2.120    0.000    2.120    0.000 :0(len)
  1261350    1.938    0.000    1.938    0.000 :0(popleft)
        2    0.000    0.000    0.001    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        4    0.000    0.000    0.000    0.000 :0(time)
  1261352    1.996    0.000    1.996    0.000 :0(values)
        1    5.629    5.629    9.823    9.823 <ipython-input-10-a42065a29c02>:15(build_trie)
        1   28.688   28.688   78.829   78.829 <ipython-input-10-a42065a29c02>:35(build_fails)
        1    0.000    0.000    0.000    0.000 <ipython-input-10-a42065a29c02>:37(<dictcomp>)
        1    0.000    0.000   88.652   88.652 <ipython-input-10-a42065a29c02>:5(__init__)
 11062364   29.204    0.000   44.246    0.000 <ipython-input-10-a42065a29c02>:76(suffixes)
        1    0.207    0.207   88.859   88.859 <string>:1(<module>)
       10    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        8    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        8    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        8    0.000    0.000    0.001    0.000 iostream.py:382(write)
       10    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        1    0.000    0.000   88.859   88.859 profile:0(AhoCorasick(genes[first:last+1], health[first:last+1]))
        0    0.000             0.000          profile:0(profiler)
       10    0.000    0.000    0.000    0.000 socket.py:334(send)
       10    0.000    0.000    0.000    0.000 threading.py:1038(_wait_for_tstate_lock)
       10    0.000    0.000    0.000    0.000 threading.py:1080(is_alive)
       10    0.000    0.000    0.000    0.000 threading.py:507(is_set)

not calling .get and only getting .fails once in suffixes

building trie from 100000 genes
build_trie took 23.723631620407104
build_fails took 133.63754796981812
         17369245 function calls in 61.788 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.000    0.000 :0(acquire)
  2522705    4.055    0.000    4.055    0.000 :0(append)
        1    0.000    0.000   61.788   61.788 :0(exec)
        8    0.000    0.000    0.000    0.000 :0(getpid)
        8    0.000    0.000    0.000    0.000 :0(isinstance)
  1261350    2.132    0.000    2.132    0.000 :0(len)
  1261350    1.950    0.000    1.950    0.000 :0(popleft)
        2    0.000    0.000    0.001    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        4    0.000    0.000    0.000    0.000 :0(time)
  1261352    1.997    0.000    1.997    0.000 :0(values)
        1    5.666    5.666    9.880    9.880 <ipython-input-17-4b10eef6d882>:15(build_trie)
        1   28.712   28.712   51.705   51.705 <ipython-input-17-4b10eef6d882>:35(build_fails)
        1    0.000    0.000    0.000    0.000 <ipython-input-17-4b10eef6d882>:37(<dictcomp>)
        1    0.000    0.000   61.586   61.586 <ipython-input-17-4b10eef6d882>:5(__init__)
 11062364   17.074    0.000   17.074    0.000 <ipython-input-17-4b10eef6d882>:62(suffixes)
        1    0.201    0.201   61.787   61.787 <string>:1(<module>)
       10    0.000    0.000    0.001    0.000 iostream.py:195(schedule)
        8    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        8    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        8    0.000    0.000    0.001    0.000 iostream.py:382(write)
       10    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        1    0.000    0.000   61.788   61.788 profile:0(AhoCorasick(genes[first:last+1], health[first:last+1]))
        0    0.000             0.000          profile:0(profiler)
       10    0.000    0.000    0.000    0.000 socket.py:334(send)
       10    0.000    0.000    0.000    0.000 threading.py:1038(_wait_for_tstate_lock)
       10    0.000    0.000    0.000    0.000 threading.py:1080(is_alive)
       10    0.000    0.000    0.000    0.000 threading.py:507(is_set)